# 1. Установка и настройка зависимостей

In [ ]:
# Установка зависимостей
!pip install selenium
!pip install gdown
!pip install chardet
!pip install fake-useragent
!pip install selenium-stealth

# Установка Google Chrome
!apt-get update
!apt-get install -y wget curl unzip

# Устанавливаем Google Chrome
!wget https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb
!dpkg -i google-chrome-stable_current_amd64.deb
!apt --fix-broken install -y

# Устанавливаем библиотеку для автоматической установки ChromeDriver
!pip install chromedriver-autoinstaller

# 2. Монтирование Google Диска и создание копии файла с ключевыми словами

In [ ]:
import gdown
import chardet

# Скачивание файла с ключевыми словами с Google Диска
file_url = 'https://drive.google.com/uc?id=***'  # Укажите ID файла из ссылки
# (Мой диск: напротив keywords.txt 3 точки - Открыть доступ - Все, у кого есть ссылка - Копировать ссылку
# - Вставить в код все, что находится в ссылке вместо *** https://drive.google.com/file/d/***/view?usp=sharing)
file_name = 'keywords.txt'

def download_keywords_file(url, output):
    print("Загружаем файл ключевых слов...")
    gdown.download(url, output, quiet=False)
    print(f"Файл сохранён как {output}")

def load_keywords(file_name):
    print("Читаем ключевые слова из файла...")
    with open(file_name, 'rb') as file:
        raw_data = file.read()
        result = chardet.detect(raw_data)
        encoding = result['encoding']
        print(f"Определённая кодировка файла: {encoding}")

    try:
        content = raw_data.decode(encoding)
        # Разделение по строкам
        keywords = [line.strip() for line in content.splitlines() if line.strip()]
        return keywords
    except Exception as e:
        print(f"Ошибка при чтении файла: {e}")
        return []

    # Загрузка ключевых слов
download_keywords_file(file_url, file_name)
keywords = load_keywords(file_name)

# 3. Загрузка и обновление списка ключевых слов

In [ ]:
# Загрузка партии ключевых слов
def load_batch(file_path, batch_size=10):
    with open(file_path, 'r', encoding='utf-8') as f:
        keywords = [k.strip() for k in f.read().split(',') if k.strip()]
    return keywords[:batch_size], keywords[batch_size:]

batch_keywords, remaining_keywords = load_batch(working_file_path)

if not batch_keywords:
    print("Все ключевые слова уже использованы.")
    exit()

# Сохраняем оставшиеся ключи обратно в файл на Google Диск
with open(working_file_path, 'w', encoding='utf-8') as f:
    f.write(', '.join(remaining_keywords))

print(f"Осталось {len(remaining_keywords)} ключевых слов.")

# 4. Настройка Selenium и Chrome для сбора данных

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium_stealth import stealth
from fake_useragent import UserAgent

# Настройки Chrome
chrome_options = Options()
chrome_options.add_argument('--headless')  # Без графического интерфейса
chrome_options.add_argument('--no-sandbox')  # Устранение ошибок среды
chrome_options.add_argument('--disable-dev-shm-usage')  # Устранение ошибок памяти
chrome_options.add_argument('--disable-blink-features=AutomationControlled')

# Указываем путь к chromedriver
service = Service()

# Генерация случайного User-Agent
ua = UserAgent()
user_agent = ua.random
chrome_options.add_argument(f'user-agent={user_agent}')

# 5. Сбор данных с CyberLeninka

In [ ]:
import time
import random
import pandas as pd
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait

# Настройки папки для сохранения
FOLDER_NAME = "TextScope"
BASE_DRIVE_PATH = "/content/drive/MyDrive"
folder_path = os.path.join(BASE_DRIVE_PATH, FOLDER_NAME)

# Проверяем и создаём папку, если её нет
if not os.path.exists(folder_path):
    os.makedirs(folder_path)

# Путь для сохранения CSV внутри папки
file_path = os.path.join(folder_path, "cyberleninka_data.csv")

# Лист для хранения данных
data = []

# Проверяем, существует ли уже файл на Google Drive
if os.path.exists(file_path):
    existing_df = pd.read_csv(file_path)
else:
    existing_df = pd.DataFrame()  # Если файл не существует, создаем пустой DataFrame

# Проход по каждому запросу
for search_term in batch_keywords:
    # Запуск браузера
    driver = webdriver.Chrome(service=service, options=chrome_options)

    # Применяем Selenium Stealth
    stealth(driver,
            languages=["ru-RU", "ru"],
            vendor="Google Inc.",
            platform="Win32",
            webgl_vendor="Intel Inc.",
            renderer="Intel Iris OpenGL Engine",
            fix_hairline=True)

    # Открываем сайт
    url = "https://cyberleninka.ru/"
    driver.get(url)

    print(f"User-Agent для запроса '{search_term}': {user_agent}")

    # Вводим запрос и выполняем поиск
    search_box = driver.find_element(By.NAME, "q")
    search_button = driver.find_element(By.CLASS_NAME, "search-btn")

    search_box.clear()
    search_box.send_keys(search_term)
    search_button.click()

    time.sleep(3)

    # Переход на первую страницу результатов
    current_page = 1

    while True:
        try:
            # Извлечение данных с текущей страницы
            list_items = driver.find_elements(By.XPATH, "//ul/li/h2/a")

            for index, item in enumerate(list_items, start=1):
                try:
                    # Переход по ссылке
                    link = item.get_attribute("href")
                    driver.get(link)

                    # Извлечение данных
                    current_url = driver.current_url
                    title = WebDriverWait(driver, 10).until(
                        EC.presence_of_element_located((By.XPATH, "/html/body/div[3]/div/span/div[2]/h1/i"))
                    ).text
                    author = driver.find_element(By.XPATH, "/html/body/div[3]/div/span/div[2]/div[1]/div[1]/ul").text
                    field_of_sciences = driver.find_element(By.XPATH, "/html/body/div[3]/div/span/div[2]/div[5]/div[2]/ul/li/a").text
                    year = driver.find_element(By.XPATH, "/html/body/div[3]/div/span/div[2]/div[5]/div[1]/div[2]/div/time").text

                    # Извлечение текста статьи
                    try:
                        text_element = WebDriverWait(driver, 10).until(
                            EC.presence_of_element_located((By.XPATH, "//div[contains(@class, 'ocr')]"))
                        )
                        extracted_text = text_element.text
                    except Exception as e:
                        extracted_text = ""

                    # Добавление данных в список
                    data.append({
                        "keyword": search_term,
                        "url": current_url,
                        "title": title,
                        "author": author,
                        "field_of_sciences": field_of_sciences,
                        "year": year,
                        "text": extracted_text
                    })

                    # Возврат к результатам поиска
                    driver.back()
                    time.sleep(random.uniform(1, 3))

                except Exception as e:
                    driver.back()

            # Переход на следующую страницу
            try:
                next_button = driver.find_element(By.XPATH, f"//a[text()='{current_page + 1}']")
                next_button.click()
                WebDriverWait(driver, 15).until(
                    EC.presence_of_element_located((By.XPATH, "//ul/li/h2/a"))
                )
                current_page += 1
                time.sleep(random.uniform(2, 5))
            except Exception as e:
                break

        except Exception as e:
            break

    driver.quit()
    time.sleep(random.uniform(2, 5))

# 6. Сохранение данных в CSV

In [ ]:
import csv

# Добавляем новые данные к старым
new_df = pd.DataFrame(data)

# Если файл существует, добавляем данные к уже существующему файлу
if not existing_df.empty:
    final_df = pd.concat([existing_df, new_df], ignore_index=True)
else:
    final_df = new_df

# Сохраняем данные в файл на Google Drive
final_df.to_csv(file_path, index=False, quoting=csv.QUOTE_ALL)
print(f"Данные успешно сохранены в {file_path}.")